In [1]:
# Data manipulation
import numpy as np
import pandas as pd

# Plotting
import plotly.express as px
import matplotlib.pyplot as plt
import streamlit as st

import datetime

#Ignore warnings
import warnings
warnings.filterwarnings("ignore")

# import quant finance libraries
import yfinance as yf

In [2]:
stocks = ['eng.mc','ele.mc', 'itx.mc', 'bbva.mc', 'vid.mc', 'rep.mc', 'ibe.mc', 'or.pa',
                'san.pa', 'azn', 'regn', 'atvi', 'msft', 'team', 'googl', 'nvda', 'csx']
start_date = datetime.date(2020, 1, 1)

In [3]:
def get_data(stocks, start_date, end_date):
            return yf.download(stocks, start = start_date, end = end_date)

In [4]:
stocks

['eng.mc',
 'ele.mc',
 'itx.mc',
 'bbva.mc',
 'vid.mc',
 'rep.mc',
 'ibe.mc',
 'or.pa',
 'san.pa',
 'azn',
 'regn',
 'atvi',
 'msft',
 'team',
 'googl',
 'nvda',
 'csx']

In [5]:
start_date

datetime.date(2020, 1, 1)

In [6]:
df = get_data(stocks, start_date = "2019-01-01", end_date = "2022-11-30")       
df = df['Adj Close']

[*********************100%***********************]  17 of 17 completed


In [7]:
def cum_returns(stocks, wts):

  weighted_returns = (wts * stocks.pct_change()[1:])
  weighted_returns = pd.DataFrame(weighted_returns)
  port_ret = weighted_returns.sum(axis=1)
  return (port_ret + 1).cumprod() 

In [8]:
total_stocks = len(stocks)
weight = [1/total_stocks]*total_stocks
#call cumulative returns
returns = cum_returns(df, weight).reset_index()
returns.rename(columns={'Date':'date', 0:'ret'}, inplace=True)

In [9]:
# optimizitation
from scipy.optimize import minimize 

def optimize_weights(returns, risk_free_return):
    
    n = returns.shape[1]
    initial_weights = np.ones(n) / n
    bounds = [(0, 1) for i in range(n)]
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    def neg_sharpe_ratio(weights, returns, risk_free_rate):
        portfolio_return = np.sum(returns.mean() * weights) * 252
        portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(returns.cov() * 252, weights)))
        sharpe_ratio = (portfolio_return - risk_free_rate) / portfolio_volatility
        return -sharpe_ratio
    result = minimize(fun=neg_sharpe_ratio, x0=initial_weights, args=(returns, risk_free_return), method='SLSQP', bounds=bounds, constraints=constraints)
    optimized_weights = result.x
    return optimized_weights

In [10]:
res = optimize_weights(df.pct_change(), 4)

In [11]:
pd.DataFrame(data=res, index=df.columns, columns=['res']).sort_values(by='res', ascending=False)[:3].index.tolist()

['NVDA', 'SAN.PA', 'ENG.MC']

In [15]:
returns = df.pct_change()
n = returns.shape[1]
weights = np.ones(n) / n
np.sum(returns.mean() * weights) * 252

0.17876103188362516

In [12]:
import tensorflow as tf

def sharpe_ratio_loss(y_true, y_pred):
    # Sharpe ratio formula
    return -tf.reduce_mean((y_true - y_pred) / tf.math.reduce_std(y_true - y_pred))

# Define the inputs
x = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0])
y = tf.constant([0.5, 1.0, 1.5, 2.0, 2.5])

# Define the model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(units=1, input_shape=[1])
])

# Compile the model
model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.01), loss=sharpe_ratio_loss)

# Train the model
history = model.fit(x, y, epochs=1000, verbose=0)

# Get the gradients
with tf.GradientTape() as tape:
    y_pred = model(x)
    loss = sharpe_ratio_loss(y, y_pred)
grads = tape.gradient(loss, model.trainable_variables)

print(grads)

[<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[-0.00019751]], dtype=float32)>, <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.00481659], dtype=float32)>]
